### Credit
simpleT5: https://github.com/Shivanandroy/simpleT5

Blogpost: https://medium.com/geekculture/simplet5-train-t5-models-in-just-3-lines-of-code-by-shivanand-roy-2021-354df5ae46ba



## Abstractive Summarization with T5 Demo

In [ ]:

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.model_selection import train_test_split

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
! pip install simplet5 -q

# Read Input Dataset - desired columns `headlines` & `text`

In [ ]:
df = pd.read_csv("../input/news-summary/news_summary.csv", encoding='latin-1', usecols=['headlines', 'text'])

In [ ]:
df.head()

# T5 Data Prep with Training Data Column Names - `source_text` & `target_text`

In [ ]:
# simpleT5 expects dataframe to have 2 columns: "source_text" and "target_text"
df = df.rename(columns={"headlines":"target_text", "text":"source_text"})
df = df[['source_text', 'target_text']]


In [ ]:
df.head()

# T5 Data Prep with Summarization Tax Prefix

In [ ]:
# T5 model expects a task related prefix: since it is a summarization task, we will add a prefix "summarize: "
df['source_text'] = "summarize: " + df['source_text']
df

In [ ]:
train_df, test_df = train_test_split(df, test_size=0.3)
train_df.shape, test_df.shape

# Using SimpleT5 for Model Training - Instantiate, Download Pre-trained Model

In [ ]:
from simplet5 import SimpleT5

model = SimpleT5()
model.from_pretrained(model_type="t5", model_name="t5-base")


# Model Training

In [ ]:
model.train(train_df=train_df[:5000],
            eval_df=test_df[:100], 
            source_max_token_len=128, 
            target_max_token_len=50, 
            batch_size=8, max_epochs=5, use_gpu=True)

# Output Folder Content


In [ ]:
! ( cd outputs; ls )

# Model Inference

In [ ]:
# let's load the trained model from the local output folder for inferencing:
model.load_model("t5","outputs/SimpleT5-epoch-2-train-loss-0.2888", use_gpu=True)



# Impressive Results

In [ ]:
#src = https://www.thehindu.com/business/Industry/twitter-interim-grievance-officer-for-india-quits/article35004295.ece
text_to_summarize="""summarize: Twitter’s interim resident grievance officer for India has stepped down, leaving the micro-blogging site without a grievance official as mandated by the new IT rules to address complaints from Indian subscribers, according to a source.

The source said that Dharmendra Chatur, who was recently appointed as interim resident grievance officer for India by Twitter, has quit from the post.

The social media company’s website no longer displays his name, as required under Information Technology (Intermediary Guidelines and Digital Media Ethics Code) Rules 2021.

Twitter declined to comment on the development.

The development comes at a time when the micro-blogging platform has been engaged in a tussle with the Indian government over the new social media rules. The government has slammed Twitter for deliberate defiance and failure to comply with the country’s new IT rules.
"""
model.predict(text_to_summarize)

In [ ]:
text_to_summarize="""summarize: Vaccination and safety measures such as wearing face masks are essential when it comes to fighting the Delta Plus coronavirus variant, World Health Organization (WHO) representative to Russia Melita Vujnovic said.

"Vaccination plus masks, because just a vaccine is not enough with 'Delta Plus'. We need to make an effort over a short period of time, otherwise there would be a lockdown," Vujnovic said on the Soloviev Live YouTube show.

She explained that vaccination is essential because it lowers the probability of spreading the virus and lowers the risks of severe disease. However, "additional measures" will probably be required as well, Vujnovic warned.

Earlier in June, the WHO included the Delta variant in its list of coronavirus variants of concern as the strain had become prevalent and has caused a resurgence of COVID-19 cases in some countries, including Russia. India has also reported multiple cases of the Delta Plus strain, which was first discovered in March.
"""
model.predict(text_to_summarize)

In [ ]:
text_to_summarize="""summarize: Travellers vaccinated with Covishield may not be eligible for the European Union’s ‘Green Pass’ that will be available for use from July 1. Many EU member states have started issuing the digital “vaccine passport” that will enable Europeans to move freely for work or tourism. The immunity passport will serve as proof that a person has been vaccinated against the coronavirus disease (Covid-19), or recently tested negative for the virus, or has the natural immunity built up from earlier infection.Covishield, a version of AstraZeneca Covid vaccine manufactured by Pune-based Serum Institute of India (SII), has not been approved by the EMA for the European market. The EU green pass will only recognise the Vaxzervria version of the AstraZeneca vaccine that is manufactured in the UK or other sites around Europe.
"""
model.predict(text_to_summarize)